In [1]:
import numpy as np
from collections import defaultdict
import copy
import itertools as it

In [2]:
def sums_idxs_dict(sum_vector):
    '''Function that returns a dictionary of the sums values and their indexes in sum vector'''
    return {k:list(np.where(sum_vector == k)[0]) for k in np.unique(sum_vector)}

In [3]:
def axis_sum_idxs_double_dict(arr1, arr2):
    '''
    Function that returns double dictionary of axis, sum value and indexes permutations from
    two arrays. It finds all the possible permutations between the arrays based on every
    value of the vector sum.
    
    Parameters
    ----------
    arr1, arr2: 2D numpy arrays
        Equal dimension arrays to be compared and to extract the permuted indexes.
        
    Returns
    -------
    s_ip: dictionary
        Dictionary of dictionary. First key is the axis. Second keys are sums values, i.e.
        integers. Values are possible permutations (2D numpy arrays) in array notation,
        therefore if the array has only one column, this column represent a new permutation
        for the case of unique sum values, and this permutation should be now permanent.
        
    Example
    -------
    >>> new_a = np.array([[0, 1, 0, 1, 1],
                          [1, 0, 1, 1, 0],
                          [1, 1, 0, 0, 0],
                          [0, 1, 1, 0, 1]])
    >>> new_b = np.array([[1, 1, 0, 0, 1],
                          [0, 1, 1, 1, 0],
                          [0, 0, 0, 1, 1],
                          [1, 0, 1, 1, 0]])
    >>> axis_sum_idxs_double_dict(new_a, new_b)
    {0: {2: array([[0, 2, 3, 4],
             [0, 1, 2, 4]]), 3: array([[1],
             [3]])}, 1: {2: array([[2],
             [2]]), 3: array([[0, 1, 3],
             [0, 1, 3]])}}
    
    To access for example the Cauchy array created by uncertainty in axis 0, this means columns
    uncertainty, and created also by uncertainty in all the columns adding up to 2, we need:
    
    >>> axis_sum_idxs_double_dict(new_a, new_b)[0][2]
    array([[0, 2, 3, 4],
           [0, 1, 2, 4]])
    '''
    
    def sums_idxspermutations_dict(arr1, arr2, ax):
        '''Function that finds all the possible permutations between the arrays based on vector
        sum values.'''
        i = 0
        keys1_list = list(sums_idxs_dict(np.sum(arr1, ax)).keys())
        keys2_list = list(sums_idxs_dict(np.sum(arr2, ax)).keys())
        assert keys1_list == keys2_list
        s_ip = {}
        sidsa1a_values = sums_idxs_dict(np.sum(arr1, ax)).values()
        sidsa2a_values = sums_idxs_dict(np.sum(arr2, ax)).values()
        for v, w in zip(sidsa1a_values, sidsa2a_values):
            assert len(v) == len(w)
            if ax == 0:
                array_value = np.concatenate((np.array([v]), np.array([w])))
            elif ax == 1:
                array_value = np.concatenate((np.array([w]), np.array([v])))
            s_ip[keys1_list[i]] = array_value
            i += 1
        return s_ip
    
    dict_of_dict = {}
    for j in (0, 1):
        dict_of_dict[j] = sums_idxspermutations_dict(arr1, arr2, ax=j)
    return dict_of_dict

In [4]:
def obtain_degeneracies_unique_only(arr1, arr2, ax, unique_sum_value, prints=False):
    '''
    Function that finds the double degeneracy for unique values in two permuted matrices and
    their associated possible permutations.
    
    Parameters
    ----------
    arr1, arr2: 2D numpy arrays
        Equal dimension arrays to be analyzed through their permuted indexes.
    ax: int
        Axis where the unique values vector is present.
    unique_sum_value: int
        Unique sum value to which its associated cauchy column must be used for locating all
        possible permutations for every matrix element type in the perpendicular dimensions.
        
    Returns
    -------
    two_arrays_dict: dict
        Dictionary of two 2D numpy arrays, one for each type of matrix element possible
        permutations. First array for matrix element and assigned key 0, second array for
        matrix element and assigned key 1. They are cauchy arrays for all the possible
        permutations in the perpendicular dimension.
        
    Example
    -------
    >>> new_a = np.array([[0, 1, 0, 1, 1],
                          [1, 0, 1, 1, 0],
                          [1, 1, 0, 0, 0],
                          [0, 1, 1, 0, 1]])
    >>> new_b = np.array([[1, 1, 0, 0, 1],
                          [0, 1, 1, 1, 0],
                          [0, 0, 0, 1, 1],
                          [1, 0, 1, 1, 0]])
                          
    The unique column adding up to 3, with original column index 1 and permuted column index 3,
    won't produce degeneracy in the rows for matrix element 0, which has original row index 1
    and permuted row index 0. But his column will produce a triple degeneracy in the rows for
    matrix element 1:
                          
    >>> obtain_degeneracies_unique_only(new_a, new_b, 0, 3)
    {0: array([[0],
            [1]]), 1: array([[1, 2, 3],
            [0, 2, 3]])}
            
    Similarly, the unique row adding up to 2, will produce the following degeneracies in the
    columns, where it is important to notice the difference in original and permuted
    convention for rows or columns Cauchy arrays:
            
    >>> obtain_degeneracies_unique_only(new_a, new_b, 1, 2)
    {0: array([[2, 3, 4],
            [0, 1, 2]]), 1: array([[0, 1],
            [3, 4]])}
    '''

    dict_of_dicts = axis_sum_idxs_double_dict(arr1, arr2)
    cauchy_column = dict_of_dicts[ax][unique_sum_value]
    if prints:
        print("cauchy_column:")
        print(cauchy_column)
    assert cauchy_column.shape == (2, 1)

    arrs = [arr1, arr2]
    zeros_idxs= []
    ones_idxs= []
    for a in (0, 1):
        if ax == 0:
            unique_vector = arrs[a][:, cauchy_column[a][0]]
        elif ax == 1:
            unique_vector = arrs[a][cauchy_column[1-a][0]]
        zeros_idxs.append(np.where(unique_vector==0)[0])
        ones_idxs.append(np.where(unique_vector==1)[0])
        if prints:
            print("\na", a)
            print("cauchy_column[1-a][0]", cauchy_column[1-a][0])
            print("unique_vector", unique_vector)
            print("np.where(unique_vector==0)[0]", np.where(unique_vector==0)[0])
            print("zeros_idxs", zeros_idxs)
            print("np.where(unique_vector==1)[0]", np.where(unique_vector==1)[0])
            print("ones_idxs", ones_idxs)
    
    two_arrays_dict = {}
    two_arrays_dict[0] = np.concatenate(([zeros_idxs[1-ax]], [zeros_idxs[ax]]))
    two_arrays_dict[1] = np.concatenate(([ones_idxs[1-ax]], [ones_idxs[ax]]))
    return two_arrays_dict

In [5]:
def partition_intersections(x, y):
    '''
    Function that finds all Cartesian product intersections between two partitions x and y
    of the same set.
    
    Parameters
    ----------
    x, y: lists
        Lists of lists, or list of 1D numpy arrays. Flattened lists of x and y should be equal.
    
    Returns
    -------
    all_s: list
        List of 1D numpy arrays of all Cartesian product intersections between x and y.
    
    Example
    -------
    >>> u = [[1, 2], [3, 4, 5], [6, 7, 8, 9, 10]]
    >>> v = [[1, 3, 6, 7], [2, 4, 5, 8, 9, 10]]
    >>> partition_intersections(u, v)
    [array([1]),
     array([2]),
     array([3]),
     array([4, 5]),
     array([6, 7]),
     array([ 8,  9, 10])] 
    
    '''
    flattened_x = np.sort(np.concatenate((x), axis=None))
    flattened_y = np.sort(np.concatenate((y), axis=None))
    assert (flattened_x == flattened_y).all()
    all_s = []
    for sx in x:
        for sy in y:
            ss = list(filter(lambda i:i in sx, sy))
            if ss:
                all_s.append(ss)#np.array(ss))
    return all_s

In [6]:
def fullaxis_uniqueonly_degeneracies_intersection(fullaxis_dict, uniqueonly_dict):
    '''
    Function that applies partition_intersections among rows or cauchy arrays.
    
    Parameters
    ----------
    fullaxis_dict: dict
        Dictionary of all sums and their indexes degeneracy for a given axis, obtained from
        axis_sum_idxs_double_dict.
    uniqueonly_dict: dict
        Dictionary of both degeneracies for given unique sum in axis perpendicular to the
        previous one, obtained from obtain_degeneracies_unique_only.
    
    Returns
    -------
    final_list: list
        List of Cauchy arrays with partially complete degeneracy for given axis.
    
    Example
    -------
    >>> new_a = np.array([[0, 1, 0, 1, 1],
                          [1, 0, 1, 1, 0],
                          [1, 1, 0, 0, 0],
                          [0, 1, 1, 0, 1]])
    >>> new_b = np.array([[1, 1, 0, 0, 1],
                          [0, 1, 1, 1, 0],
                          [0, 0, 0, 1, 1],
                          [1, 0, 1, 1, 0]])
                          
    If we want to find intersection of degeneracies due to all columns (axis 0), knowing the
    degeneracies obtained due to rows (axis 1) with unique sum value 2, the respective Cauchy
    arrays for columns are:
    
    >>> asiddnanb0 = axis_sum_idxs_double_dict(new_a, new_b)[0]
    >>> oduonanb12 = obtain_degeneracies_unique_only(new_a, new_b, 1, 2)
    >>> fullaxis_uniqueonly_degeneracies_intersection(asiddnanb0, oduonanb12)
    [array([[2, 3, 4],
            [0, 1, 2]]), array([[0],
            [4]]), array([[1],
            [3]])]
    '''
    
    def dic_row(dic, row):
        '''Function that contructs list of 1D numpy arrays (rows) from row of dictionary of
        2D numpy arrays'''
        return [arr[row] for arr in dic.values()]
    
    pdp0 = partition_intersections(dic_row(fullaxis_dict, 0), dic_row(uniqueonly_dict, 0))
    pdp1 = partition_intersections(dic_row(fullaxis_dict, 1), dic_row(uniqueonly_dict, 1))
    final_list = []
    for m, n in zip(pdp0, pdp1):
        final_list.append(np.concatenate(([m], [n])))
    return final_list

In [7]:
def unique_sum_values(sum_vector):
    '''Function that returns the unique sum values for a given (row or column) sum vector'''
    sid = sums_idxs_dict(sum_vector)
    return [k for k, v in sid.items() if len(v)==1]

In [8]:
def FINAL(a, b, prints=False):
    partial_permutations_dict = {}
    for j in (1, 0):
        us_a = unique_sum_values(np.sum(a, axis=j))
        us_b = unique_sum_values(np.sum(b, axis=j))
        if prints:
            print("\nj", j)
            print(np.sum(a, axis=j), "us_a:", us_a)
            print(np.sum(b, axis=j), "us_b:", us_b)
        assert us_a == us_b
        my_dict = {}
        c = 0
        p_p = []
        for u in us_a:
            asiddabj = axis_sum_idxs_double_dict(a, b)[1-j]
            oduoabju = obtain_degeneracies_unique_only(a, b, j, u)
            fa_uo_di = fullaxis_uniqueonly_degeneracies_intersection(asiddabj, oduoabju)
            if prints:
                print("u", u)
                print("fa_uo_di:")
                print(fa_uo_di)
            my_dict[u] = fa_uo_di
            c += 1
            if c == 1:
                p_p = my_dict[us_a[c-1]]
            if c > 1:
                t_dict = {}
                for i in (0, 1):
                    rows_minus1 = [sublist[i] for sublist in my_dict[us_a[c-1]]]
                    rows_partial = [sublist[i] for sublist in p_p]
                    t_dict[i] = partition_intersections(rows_minus1, rows_partial)
                p_p = [np.concatenate(([m], [n])) for m, n in zip(t_dict[0], t_dict[1])]
            if prints:
                print("p_p", p_p)
        if p_p:
            partial_permutations_dict[1-j] = p_p
        else:
            asiddabj = axis_sum_idxs_double_dict(a, b)[1-j]
            partial_permutations_dict[1-j] = [v for v in asiddabj.values()]

    return partial_permutations_dict

In [9]:
def ready_nonready_arrays(new_a, new_b, ab_dict, prints=False, prints_counter=False):
    ready_arrays_dict = {}
    nonready_arrays_dict = {}
    for k, v in ab_dict.items():
        nonready_arrays = []
        if prints:
            print("\nk", k)
            print("v", v)
        r = n = 0
        for w in v:
            if w.shape[1] == 1:
                r += 1
                if prints:
                    print(r, "READY:")
                    print(w)
                if r == 1:
                    ready_array = w
                    if prints:
                        print("ready_array:")
                        print(ready_array)
                else:
                    ready_array = np.concatenate((ready_array, w), axis=1)
                    if prints:
                        print("ready_array:")
                        print(ready_array)
            else:
                n += 1
                if prints:
                    print(n, "not ready:")
                    print(w)
                    print("nonready_arrays", nonready_arrays)
                nonready_arrays.append(w)
        if prints_counter:
            print("Final counters (r: ready, n: not ready)")
            print("r:", r)
            print("n:", n)
        if r != 0:
            ready_arrays_dict[k] = ready_array
        if n != 0:
            nonready_arrays_dict[k] = nonready_arrays
    return ready_arrays_dict, nonready_arrays_dict

In [10]:
def permuted_order_cauchy_to_python(arr, ax):
    '''
    Example
    -------
    >>> cauchy = np.array([[1, 0, 2, 3, 5],
                           [0, 2, 3, 1, 5]]) 
    >>> poctpC = permuted_order_cauchy_to_python(cauchy, 1)
    >>> poctpC    
    array([2, 0, 3, 1, 5])
    >>> poctpR = permuted_order_cauchy_to_python(cauchy, 0)
    >>> poctpR
    array([1, 3, 0, 2, 5])
    '''
    cauchy_order = arr[:, np.argsort(arr[1-ax])]
    python_permutation_order = cauchy_order[ax]    
    return python_permutation_order

# ITERATIVE FUNCTION:

In [11]:
def pythonic(s):
    all_elements = defaultdict(list)
    for i, ss in enumerate(s):
        for elem in ss:
            all_elements[elem].append(i)
    reversed = defaultdict(list)
    for k, v in all_elements.items():
        reversed[frozenset(v)].append(k)
    return reversed#.keys()#.values()#list(reversed.values())

In [12]:
def update_dicts(to_update_ready_dict, to_update_nonready_dict, AA, BB, prints=False):
    initial_ready_dict = copy.deepcopy(to_update_ready_dict)
    tup = (to_update_nonready_dict[0][0], to_update_nonready_dict[1][0])
    A = AA[tup[1][1]][:, tup[0][0]]
    B = BB[tup[1][0]][:, tup[0][1]]
    r_n_tuple = ready_nonready_arrays(A, B, FINAL(A, B))
    ready_AB_dict = r_n_tuple[0]
    nonready_AB_dict = r_n_tuple[1]
    ready_sub_dict = {k: tup[k][np.arange(2)[:, None], ready_AB_dict[k]] for k in 
                      ready_AB_dict.keys()}
    nonready_sub_dict = {k: [tup[k][np.arange(2)[:, None], v] for v in 
                             nonready_AB_dict[k]] for k in nonready_AB_dict.keys()}
    if prints:
        print("tup", tup)
        print("A:")
        print(A)
        print("B:")
        print(B)
        print("ready_sub_dict", ready_sub_dict)
        print("nonready_sub_dict", nonready_sub_dict)

    for ax in ready_sub_dict:
        if prints:
            print("AXIS ax", ax, "analysis for ready_sub_dict:")
            print("initial to_update_ready_dict[ax]", to_update_ready_dict[ax])
            print("ready_sub_dict[ax]", ready_sub_dict[ax])
        for col in ready_sub_dict[ax].T:
            cauchy_col = np.reshape(col, (2, 1))
            if not (to_update_ready_dict[ax] == cauchy_col).all(axis=0).any():
                to_update_ready_dict[ax] = np.concatenate((to_update_ready_dict[ax],
                                                           cauchy_col), axis=1)
                if prints:
                    print("to_update_ready_dict[ax]:")
                    print(to_update_ready_dict[ax])
                    print("initial to_update_nonready_dict[ax]", to_update_nonready_dict[ax])
                nr0_bool = [cauchy_col[0] in arr[0] for arr in to_update_nonready_dict[ax]]
                nr1_bool = [cauchy_col[1] in arr[1] for arr in to_update_nonready_dict[ax]]
                assert nr0_bool.index(True) == nr1_bool.index(True)
                true_arr_idx = nr0_bool.index(True)
                arr_to_update = to_update_nonready_dict[ax][true_arr_idx]
                #print("initial arr_to_update", arr_to_update)
                idxs = [(i, np.where(arr_to_update[i]==cauchy_col[i])[0][0]) for i in (0, 1)]
                #print("idxs", idxs)
                atus0 = arr_to_update.shape[0]
                atus1 = arr_to_update.shape[1]
                idxs = [i*atus1+j for i, j in idxs]
                #print("idxs", idxs)
                arr_to_update = np.delete(arr_to_update, idxs).reshape(atus0, atus1-1)
                #print("final arr_to_update", arr_to_update)
                if arr_to_update.shape == (2, 1):
                    to_update_ready_dict[ax] = np.concatenate((to_update_ready_dict[ax],
                                                               arr_to_update), axis=1)
                    to_update_nonready_dict[ax].pop(true_arr_idx)
                    if prints:
                        print("if, to_update_ready_dict instead:")
                        print("to_update_ready_dict[ax]", to_update_ready_dict[ax])
                        print("to_update_nonready_dict[ax]", to_update_nonready_dict[ax])
                else:
                    to_update_nonready_dict[ax][true_arr_idx] = arr_to_update
                    if prints:
                        print("else, to_update_nonready_dict[ax] now:")
                        print("to_update_nonready_dict[ax]", to_update_nonready_dict[ax])

    for ax in nonready_sub_dict:
        if prints:
            print("AXIS ax", ax, "analysis for nonready_sub_dict:")
            print("initial to_update_nonready_dict[ax]", to_update_nonready_dict[ax])
            print("nonready_sub_dict[ax]", nonready_sub_dict[ax])
        nrdax0 = [a[0] for a in to_update_nonready_dict[ax]]
        nrsdax0 = [a[0] for a in nonready_sub_dict[ax]]
        nrdax1 = [a[1] for a in to_update_nonready_dict[ax]]
        nrsdax1 = [a[1] for a in nonready_sub_dict[ax]]
        PDP0 = [v for v in pythonic(nrdax0 + nrsdax0).values()]
        PDP1 = [pythonic(nrdax1 + nrsdax1)[k] for k in pythonic(nrdax0 + nrsdax0).keys()]
        new_list = []
        for m, n in zip(PDP0, PDP1):
            new_list.append(np.concatenate(([m], [n])))
        idx_to_pop = []
        for c, arr in enumerate(new_list):
            if ((arr.shape == (2, 1)) and
                not (to_update_ready_dict[ax] == arr).all(axis=0).any()):
                to_update_ready_dict[ax] = np.concatenate((uto_update_ready_dict[ax], arr),
                                                          axis=1)
                if prints:
                    print("To update in to_update_ready_dict[ax] the arr:", arr)
                    print("to_update_ready_dict[ax]:")
                    print(to_update_ready_dict[ax])
                idx_to_pop.append(c)
            elif (arr.shape == (2, 1)) and (to_update_ready_dict[ax] == arr).all(axis=0).any():
                if prints:
                    print("Already updated in to_update_ready_dict[ax] the arr:", arr)
                idx_to_pop.append(c)
        for i, idx in enumerate(idx_to_pop):
            new_list.pop(idx - i) #because when poping an index, it changes the future indexes
        to_update_nonready_dict[ax] = new_list
        if prints:
            print("to_update_nonready_dict[ax]", to_update_nonready_dict[ax])

    breaker = False
    if (np.array_equal(initial_ready_dict[0], to_update_ready_dict[0]) and
        np.array_equal(initial_ready_dict[1], to_update_ready_dict[1])):
        if prints:
            print("There was no change in the last iteration therefore procedure stops here")
        breaker = True
    return to_update_ready_dict, to_update_nonready_dict, breaker

In [13]:
def blablabla(A, B, prints=False):
    ready_nonready_tuple = ready_nonready_arrays(A, B, FINAL(A, B))
    ready_dict = ready_nonready_tuple[0]
    nonready_dict = ready_nonready_tuple[1]
    if prints:
        print("ready_dict", ready_dict)
        print("nonready_dict", nonready_dict)
        print("\n")
        
    if (0 in nonready_dict) and (1 in nonready_dict):
        j = 0
        while (nonready_dict[0]) and (nonready_dict[1]):
            ready_dict, nonready_dict, br = update_dicts(ready_dict, nonready_dict, A, B,
                                                         prints=prints)
            if prints:
                print("AND THE", j, "th nonready_dict OBTAINED IS", nonready_dict)
            if br == True:
                break
            j += 1

    return ready_dict, nonready_dict

In [14]:
def all_indexes_permutations(arr):
    '''
    Example
    -------
    >>> all_indexes_permutations(np.reshape(np.arange(6), (2,3)))
    array([[[0, 1, 2],
            [3, 4, 5]],

           [[0, 1, 2],
            [3, 5, 4]],

           [[0, 1, 2],
            [4, 3, 5]],

           [[0, 1, 2],
            [4, 5, 3]],

           [[0, 1, 2],
            [5, 3, 4]],

           [[0, 1, 2],
            [5, 4, 3]]])
    '''
    assert arr.shape[0] == 2
    cases_number = np.math.factorial(arr.shape[1])
    multi_array = np.zeros((cases_number, arr.shape[0], arr.shape[1]), dtype=int)
    j = 0
    for i in it.permutations(arr[1]):
        new_array = np.stack((arr[0], i))
        multi_array[j] = new_array
        j += 1
    return multi_array

## Testing
## Input dimensions of random matrix to create (rows, cols): dim
## Input maximum number of brute force permutations to try: max_perm

In [15]:
dim = (97, 101)
max_perm = 200000 # less than 5 minutes in my laptop, try 50000 for fast results

### The (random) input array a and its random permutation b (if you get an error, restart from here):

In [29]:
a = np.random.randint(2, size=dim)
ar = np.arange(a.shape[0])
np.random.shuffle(ar)
ac = np.arange(a.shape[1])
np.random.shuffle(ac)
b = a[ar][:,ac]

In [30]:
a

array([[1, 1, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 1, 0, 1],
       [1, 1, 1, ..., 1, 1, 1],
       ...,
       [1, 0, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 0, 1],
       [1, 1, 1, ..., 1, 0, 1]])

In [31]:
b

array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 1, 1],
       [0, 0, 0, ..., 1, 0, 0],
       ...,
       [0, 0, 0, ..., 1, 1, 1],
       [1, 0, 0, ..., 1, 1, 0],
       [1, 1, 1, ..., 0, 1, 1]])

#### NOW we execute the full algorithm:

In [32]:
urd, unrd = blablabla(a, b)#, prints=True)

##### For the row and colum indexes where we still have a degeneracy, a brute force approach is implemented, where every row in the cauchy array contributes with the factorial of its number of columns to the total, and it is calculated beforehand in brute_force_permutations

In [33]:
col_dims = [arr.shape[1] for arr in [a for sublist in unrd.values() for a in sublist]]
brute_force_permutations = np.prod(col_dims)
brute_force_permutations

165888

Finally, when implementing the brute force permutations of the remaining degenerate indexes, the (unique) counter index where the permutation between a and b is found is called successful_permutation_counter

In [34]:
if brute_force_permutations < max_perm:

    aip0 = [all_indexes_permutations(arr) for arr in unrd[0]]
    aip1 = [all_indexes_permutations(arr) for arr in unrd[1]]

    c = 0
    for j in it.product(*aip0):
        perm0 = np.concatenate(j, axis=-1)
        #print(perm0)
        funrd = {}
        for i in it.product(*aip1):
            c += 1
            #print("c", c)
            perm1 = np.concatenate(i, axis=-1)
            #print(perm1)
            funrd[0] = perm0
            funrd[1] = perm1
            #print("funrd", funrd)
            d2 = [urd, funrd]
            d = {}
            for k in urd.keys():
                if k in funrd.keys():
                    d[k] = np.hstack([dd[k] for dd in d2])
                else:
                    d[k] = urd[k]
            poctpd11 = permuted_order_cauchy_to_python(d[1], 1)
            poctpd00 = permuted_order_cauchy_to_python(d[0], 0)
            p_a = a[poctpd11][:, poctpd00]
            if (p_a == b).all():
                print("SUCCESS!!!")
                successful_permutation_counter = c
                print("successful_permutation_counter:", successful_permutation_counter)
                rows_permutation = permuted_order_cauchy_to_python(d[1], 1)
                cols_permutation = permuted_order_cauchy_to_python(d[0], 0)
    print("Total number of performed brute force permutations:", c)
    
else:
    print("Restart the matrices, the number of permutations is too large")

successful_permutation_counter: 1057490
Total number of performed brute force permutations: 2654208


# Finally, the required permutations in python indexing style are:

In [35]:
rows_permutation

array([44, 81, 61, 40, 12, 74,  8, 25, 13, 47, 68, 41, 59, 46, 83, 95, 22,
       75, 33, 65, 63, 38, 52, 88, 48,  0, 92, 42, 35, 14, 39, 37, 69, 20,
       85, 15,  9, 57, 19, 24, 43, 29, 55, 58, 77, 27, 73,  3,  7,  6,  4,
       79, 34, 76, 56, 10,  5, 51, 31, 84, 53, 93, 17, 90, 18, 26, 94, 21,
       50, 67, 91,  1, 86, 82, 36, 62, 60, 96, 78, 32, 89, 16, 71, 80,  2,
       49, 11, 23, 64, 54, 30, 87, 45, 28, 70, 72, 66])

In [36]:
cols_permutation

array([ 57,  43,  53,  97,  85,  76,  27,  47,  15,  63,  93,  38,  19,
        58,  21,  60,  36,  42,  40,  96,  35,  78,  91,  16,   9,  73,
        82,  24,   2,  84,  98,  66,  32,  10, 100,  20,  89,   4,  67,
         7,  69,  23,  41,  26,  39,  64,  14,  29,  75,   0,  77,  88,
        44,  72,  48,   5,  54,  11,  56,  90,  80,  30,  18,  37,  92,
        22,  62,  33,  50,  71,  61,  13,  94,  70,  74,  86,  34,  87,
        99,  68,  55,  45,  51,  65,  17,  81,   3,  49,  59,  83,  25,
         1,   8,  95,   6,  52,  28,  79,  31,  46,  12])